In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('./data/dataset.csv')
df

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%)
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
201,2l-c,[*]C1=CC=CC=C1,Ph,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,72.0,0.0,0.0,92.0
202,2m,[*]C1=CSC=C1,C,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,83.0
203,2n,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,53.0
204,2o,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,25.0


In [24]:
# 基質、触媒、収率で欠損がある場合は削除
df.dropna(subset=['R1-', 'R2', 'organocatalyst', '収率(%)'], inplace=True)

In [25]:
print('欠損値合計', df.isnull().sum())
na_row = df.isnull().any(axis=1)
df.loc[na_row, :]

欠損値合計 entry                   0
R1-                     0
R2                      0
organocatalyst          0
organocatalyst(mol%)    0
solvent                 0
under                   0
temp(℃)                 0
time(h)                 0
Cu(OAc)2(mol%)          0
AcOH(mol%)              0
収率(%)                   0
dtype: int64


,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%)


In [26]:
# Arを削除(1 data)
df = df[df['under'] != 'Ar']
df['under'].value_counts()

O2     110
air     44
Name: under, dtype: int64

In [27]:
# R2 = Ph削除(3 data)
df = df[df['R2'] != 'Ph']
df['R2'].value_counts()

H    99
C    52
Name: R2, dtype: int64

In [28]:
# 触媒促進剤データ削除
df = df[df['Cu(OAc)2(mol%)'] == 0]
df = df[df['AcOH(mol%)'] == 0]
print(df.columns)
print(df.shape)

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'Cu(OAc)2(mol%)',
       'AcOH(mol%)', '収率(%)'],
      dtype='object')
(123, 12)


In [31]:
# 触媒カラム削除
df.drop(columns=['Cu(OAc)2(mol%)','AcOH(mol%)'],inplace=True)

df.columns

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', '収率(%)'],
      dtype='object')

In [32]:
# rtを23度に置換
df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"
print(df['temp(℃)'].value_counts())

23    65
90    49
80     7
60     2
Name: temp(℃), dtype: int64


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_86251/2448397563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"


In [34]:
# カテゴリ値をダミー変数に変換
df = pd.get_dummies(df, columns=['R2','under','solvent'])
df.head(5)

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),収率(%),R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_DMF,solvent_MeCN,solvent_MeOH,solvent_toluene
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,24.0,81.0,0,1,1,0,0,0,0,0,1
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,12.0,14.0,0,1,1,0,0,0,0,0,1
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,24.0,74.0,0,1,1,0,0,0,0,0,1
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,12.0,15.0,0,1,1,0,0,0,0,0,1
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,5.0,90,24.0,90.0,0,1,1,0,0,0,0,0,1


In [ ]:
# 重複確認
check_columns = df.columns.drop(['entry', '収率(%)'])

# 説明変数のみでみた時の重複削除
_df = df.drop(columns='entry').drop_duplicates(subset=check_columns).reset_index(drop=True)
X = _df.drop(columns=['収率(%)'])
y = _df['収率(%)']
_df.to_csv('exec_1114_前処理後.csv')

In [42]:
check_columns = df.columns.drop(['entry', '収率(%)'])
df.duplicated(subset=check_columns).value_counts()

False    123
dtype: int64

In [43]:
df.to_csv('./data/origin.csv')

In [45]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)']
scaler = StandardScaler()
scaler.fit(df[scaling_columns])
df_scaled = pd.concat([df.drop(columns=scaling_columns), 
                           pd.DataFrame(scaler.transform(df[scaling_columns]),index=df.index , columns=scaling_columns)],
                           axis=1, join='inner')

print(df_scaled.shape)

(123, 16)


In [48]:
df_scaled.to_csv('./data/origin_scaled.csv')